# MIMICモデル & PLSモデル

## MIMICモデル

**MIMIC (Multiple Indicator Multiple Cause) モデル**は、観測変数をまとめた合成変数を観測変数で説明するという、回帰分析と因子分析を合わせたようなモデル。

### 歴史

経済学者のArthur Goldbergerが統計学者との共著で1975年に提案



### 参考

[Charles F. Manski, "Arthur S. Goldberger", Biographical Memoirs of the National Academy of Sciences (2013)](http://www.nasonline.org/publications/biographical-memoirs/memoir-pdfs/goldberger-arthur.pdf)


GoldbergerはMIMICモデルだけでなく、"mean independence"の提案やSelection Biasに関する研究など因果推論への貢献も多い


## PLSモデル (Partial Least Square Model)

MIMICモデルにおける合成変数を別の因子の説明に使ったもの。

観測変数をまとめた合成変数から、別の複数の観測変数の潜在変数に与える影響の強さを検討するモデル